### Imports and Setup

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib
import datetime

from tensorflow.keras.models import model_from_json

### Data Preprocessing

In [ ]:
IMG_SIZE = 224
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE=1000
TEST_SIZE=1000
VAL_SIZE=1000

In [ ]:
# Use my dataset
data_dir = "/home/ubuntu/cs230/UTKFace/*_*_*_*"
# list_files lists files in a shuffled order
list_ds = tf.data.Dataset.list_files(data_dir, seed=8)

# Convert data_dir to a Path object and count images
data_dir = pathlib.Path(data_dir[:-7])
image_count = len(list(data_dir.glob('*')))

In [ ]:
# Define helper functions

def get_label(file_path):
    parts_1 = tf.strings.split(file_path, "/")
    parts = tf.strings.split(parts_1[-1], "_")
#     if len(parts)>3:
#         parts = '0'
    return int(parts[0])
    

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
    return tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

def process_path(file_path):
    label = get_label(file_path)
  # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

test_batches = labeled_ds.take(TEST_SIZE).batch(BATCH_SIZE)
validation_batches = labeled_ds.skip(TEST_SIZE).take(VAL_SIZE).batch(BATCH_SIZE)
train_batches = labeled_ds.skip(TEST_SIZE + VAL_SIZE).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def false_pos (y_true, y_pred):
#     ans = (y_pred <= 15 and y_true >= 15)
    ans = (1-tf.math.maximum(0.0,y_pred-15)/(y_pred-15 + 0.001))*(tf.math.maximum(0.0,y_true-15)/(y_true-15+0.001))
    return ans

In [ ]:
def false_neg (y_true, y_pred):
#     ans = (y_pred >= 15 and y_true <= 15)
    ans = (tf.math.maximum(0.0,y_pred-15)/(y_pred-15 + 0.001))*(1-tf.math.maximum(0.0,y_true-15)/(y_true-15+0.001))
    return ans

In [ ]:
def true_neg (y_true, y_pred):
#     ans = (y_pred >= 15 and y_true >= 15)
    ans = (tf.math.maximum(0.0,y_pred-15)/(y_pred-15+0.001))*(tf.math.maximum(0.0,y_true-15)/(y_true-15+0.001))
    return ans

In [ ]:
def true_pos (y_true, y_pred):
#     ans = (y_pred <= 15 and y_true <= 15)
    ans = (1-tf.math.maximum(0.0,y_pred-15)/(y_pred-15+0.001))*(1-tf.math.maximum(0.0,y_true-15)/(y_true-15+0.001))
    return ans

In [ ]:
def minor_accuracy (y_true, y_pred):
    ans = true_neg(y_true,y_pred)+true_pos(y_true,y_pred)
    return ans

In [ ]:
## Just for a sanity check
def combined (y_true, y_pred):
    ans = true_pos(y_true,y_pred)+false_pos(y_true,y_pred)+false_neg(y_true,y_pred)+true_neg(y_true,y_pred)
    return ans

In [ ]:
model = tf.keras.models.load_model('/home/ubuntu/cs230/saved_models/final_model', compile=False)

### Model Evaluation

In [ ]:
model.compile(metrics=['accuracy', tf.metrics.RootMeanSquaredError(), false_neg, false_pos, true_neg, true_pos, minor_accuracy])

In [ ]:
loss0, accuracy0, rmse0, false_neg0, false_pos0, true_neg0, true_pos0, minor_accuracy0 = model.evaluate(test_batches)

In [ ]:
precision0 = precision(true_pos0, false_pos0)
recall0 = recall(true_pos0, false_neg0)
fOne0 = fOne(precision0, recall0)

print("Precision: ", precision0)
print("Recall: ", recall0)
print("F1 Score: ", fOne0)
print("RMSE: ", rmse0)
print("Minor Accuracy: ", minor_accuracy0)